# ლექცია #13

## კოდების გაშვებისთვის საჭირო ბიბლიოთეკების იმპორტები

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## პერსეპტრონი (Perceptron)

პერსეპტრონი არის supervised სწავლების ორობითი კლასიფიკაციის ალგორითმი. ის მიიჩნევა ხელოვნური ნეირონული ქსელის (Artificial Neural Network) ერთეულად.

* **ერთშრიანი პერსეპტრონი** (**Single Layer Perceptron**): შეუძლია მხოლოდ წრფივად განცალკევებადი შაბლონების სწავლა
* **მრავალშრიანი პერსეპტრონი** (**Multi-Layer Perceptron**): შეუძლია კომპლექსური შაბლონების სწავლა

![A biological neuron in comparison to an artificial neural network](https://www.researchgate.net/publication/339446790/figure/fig2/AS:862019817320450@1582532948784/A-biological-neuron-in-comparison-to-an-artificial-neural-network-a-human-neuron-b.png)

![ANN](https://miro.medium.com/v2/resize:fit:1100/format:webp/1*qQPpdtR0r1APiEfTqN74aA.png)

მათემატიკურად წრფივი მოდელი შეგვიძლია წარმოვადგინოთ შემდეგი ფუნქციის სახით:

$$f(\mathbf {x}) = \sum^{n}_{i=1} w_ix_i + b = w_1x_1 + w_2x_2 + ... + w_nx_n + b = \mathbf {w}  \cdot \mathbf {x} + b$$

აქტივაციის ფუნქცია (Unit Step Function) კი:

$$g(\mathbf {z}) = \begin{cases} 1 & \mbox{if } \mathbf {z} \ge \theta \\ 0 & otherwise \end{cases}$$

<div style="width:30%; margin: auto;">
<img src="https://miro.medium.com/max/762/1*K9QJmeG33SvQeJgPa52mmQ.png" alt="Unit Step Function"/>
</div>

შესაბამისად, ფორმულას ექნება შემდეგი სახე:

$$\hat{y} = g(f(\mathbf {x})) = g(\mathbf {w}  \cdot \mathbf {x} + b)$$

**პერსეპტრონის განახლების წესი**

თითოეული სატრენინგო მაგალისთისთვის ($x_i$):

$$w = w + \Delta w$$

$$b = b + \Delta b$$

სადაც,

$$\Delta w = \alpha \cdot (y_i - \hat{y_i}) \cdot x_i$$

$$\Delta b = \alpha \cdot (y_i - \hat{y_i})$$

სადაც $\alpha$ არის learning rate (ხანდახან მოიხსენიება ხოლმე, როგორც ნაბიჯის ზომა - step size), რომელიც უმეტესად მოთავსებულია 0-სა და 1-ს შორის.

In [2]:
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.activation_func = lambda x: np.where(x > 0, 1, 0)
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        _, n_features = X.shape

        # თავდაპირველი წონები და bias
        self.weights = np.zeros(n_features)
        self.bias = 0

        # პარამეტრების (წონებისა და bias-ის) სწავლა
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                lin_out = np.dot(x_i, self.weights) + self.bias
                y_hat = self.activation_func(lin_out)

                update = self.learning_rate * (y[idx] - y_hat)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        lin_out = np.dot(X, self.weights) + self.bias
        y_hat = self.activation_func(lin_out)

        return y_hat

In [3]:
breast_cancer_data = load_breast_cancer()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    breast_cancer_data.data,
    breast_cancer_data.target,
    test_size=0.2,
    stratify=breast_cancer_data.target,
    random_state=2,
)

In [5]:
print("სატრენინგო მონაცემების ზომა:", X_train.shape, y_train.shape)
print("სატესტო მონაცემების ზომა:", X_test.shape, y_test.shape)

სატრენინგო მონაცემების ზომა: (455, 30) (455,)
სატესტო მონაცემების ზომა: (114, 30) (114,)


In [6]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

In [7]:
train_preds = perceptron.predict(X_train)
test_preds = perceptron.predict(X_test)

print(
    "აკურატულობა სატრენინგო მონაცემებზე: "
    f"{accuracy_score(y_train, train_preds) * 100:.2f}%"
)
print(
    "აკურატულობა სატესტო მონაცემებზე: "
    f"{accuracy_score(y_test, test_preds) * 100:.2f}%"
)

აკურატულობა სატრენინგო მონაცემებზე: 92.53%
აკურატულობა სატესტო მონაცემებზე: 91.23%


## დამატებითი მასალები

`scikit-learn`-ში არსებობს [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) და [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) კლასები, რომლებიც რეალურად მრავალშრიანი პერსეპტრონის იმპლემენტაციაა. თუმცა როდესაც ვმუშაობთ ნეირონულ ქსელებზე, `scikit-learn` ბიბლიოთეკას არ ვიყენებთ, რადგანაც ის მხოლოდ CPU-ზე მუშაობს და არ აქვს უამრავი ფუნქციონალის მხარდაჭერა, რაც საჭიროა ნეირონული ქსელების შესაქმნელად. ამიტომაც ხშირად გამოიყენება ისეთი ფრეიმვორქები, როგორებიცაა [PyTorch](https://pytorch.org/) და [TensorFlow](https://www.tensorflow.org/).


ძალიან საინტერესო რამდენიმე ვიდეო აქვს [3blue1brown](https://www.3blue1brown.com/topics/neural-networks)-ს ნეირონულ ქსელებზე და გირჩევთ რომ უყუროთ.